In [1]:
import psycopg2, matplotlib.pyplot as plt, io, base64, plotly
from flask import Flask, render_template, request, url_for, redirect
import pandas as pd
import plotly.express as px
import plotly
import json

app = Flask(__name__)

def bd(sql):
    conn = psycopg2.connect(host='localhost', database='IPA1', user='postgres', password = '150504')
    cur = conn.cursor()
    cur.execute(sql)
    dados = cur.fetchall()
    cur.close()
    conn.close()
    return dados

@app.route('/', methods=('GET', 'POST'))
def autentic():
    if request.method == 'POST':
        conta = request.form['login']
        senha = request.form['senha']
        sql="select * from administrador.usuario where (login='"+conta+"' and senha='"+senha+"')"
        #sql="select * from administrador.usuario where (login='"+conta+"')"
        record=bd(sql)
        nr=len(record)
        if nr>=1:
            return redirect(url_for('menu'))
        else:
            return redirect(url_for('indexnulo'))
    return render_template('autentic.html')

@app.route('/nulo')
def indexnulo():
    return render_template('indexnulo.html')

@app.route('/t1')
def menu():
    sql='select * from menu'
    menus= bd(sql)
    return render_template('menu.html', menus=menus)

@app.route('/t2')
def submenu():
    sql='select * from submenu'
    menus= bd(sql)
    rota_por_nome = {
        'crimes por cidade': 'crimes_por_cidade',
        'crimes': 'crimes_por_nome',
        'faccionado': 'criminosos_faccionados'
    }

    return render_template('submenu.html', menus=menus, 
                           rota_por_nome=rota_por_nome)

@app.route('/bi')
def index2():
    return render_template('index2.html')

@app.route('/dashboard/crimes_por_cidade')
def crimes_por_cidade():
    sql = '''SELECT cidade, COUNT(*) 
    FROM ocorrencia 
    GROUP BY cidade 
    '''
    df = bd(sql)
    cidades = [linha[0] for linha in df]
    totais = [linha[1] for linha in df]

    
    df = pd.DataFrame({'Cidade': cidades, 'Total de Ocorrências': totais})

    fig = px.bar(df, x='Cidade', y='Total de Ocorrências',
                 title='Total de Ocorrências por Cidade',
                 color='Total de Ocorrências',
                 color_continuous_scale='Blues')

    fig.update_layout(xaxis_tickangle=-45)

    graph_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return render_template("crimes_por_cidade.html", graphJSON=graph_json)

@app.route('/dashboard/crimes_por_nome')
def crimes_por_nome():
    sql = '''
        SELECT c.nome_crime, COUNT(co.id_ocorrencia) AS total_ocorrencias
        FROM crime c
        JOIN crime_ocorrencia co ON c.id_crime = co.id_crime
        GROUP BY c.nome_crime
        ORDER BY total_ocorrencias DESC;
    '''
    df = bd(sql)
    nomes_crimes = [linha[0] for linha in df]
    totais = [linha[1] for linha in df]

    df = pd.DataFrame({
        'Crime': nomes_crimes,
        'Total de Ocorrências': totais
    })

    fig = px.bar(df, 
                 x='Crime', 
                 y='Total de Ocorrências',
                 title='Quantidade de Ocorrências por Tipo de Crime',
                 color='Total de Ocorrências',
                 color_continuous_scale='Viridis')

    fig.update_layout(xaxis_tickangle=-45)

    graph_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

    return render_template('crimes_por_nome.html', graphJSON=graph_json)

@app.route('/dashboard/criminosos_faccionados')
def criminosos_faccionados():
    sql = '''
        SELECT faccionado, COUNT(*)
        FROM criminoso
        GROUP BY faccionado 
    '''
    df = bd(sql)  # lista de tuplas [(faccionado, count), ...]

    labels = [linha[0] for linha in df]
    counts = [linha[1] for linha in df]

    df_plot = pd.DataFrame({'Faccionado': labels, 'Quantidade': counts})

    fig = px.bar(df_plot, x='Faccionado', y='Quantidade',
                 title='Quantidade de Criminosos Faccionados vs Não Faccionados',
                 color='Quantidade',
                 color_continuous_scale='Blues')

    fig.update_layout(xaxis_tickangle=-45)

    graph_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
    return render_template("criminosos_faccionados.html", graphJSON=graph_json)


if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Jul/2025 14:32:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2025 14:32:26] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [06/Jul/2025 14:32:27] "GET /t1 HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2025 14:33:01] "GET /t2 HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2025 14:33:09] "GET /dashboard/crimes_por_cidade HTTP/1.1" 200 -
127.0.0.1 - - [06/Jul/2025 14:33:29] "GET /dashboard/criminosos_faccionados HTTP/1.1" 200 -
